# Extract Papers

The code is from [OpenAlex_API_Tutorial](https://github.com/ourresearch/openalex-api-tutorials/blob/1988d22c5499d6a1f68d85ef2902b600b555aaa5/notebooks/getting-started/api-webinar-apr2024/tutorial01.ipynb)

In [ ]:
import requests
import time
import csv
import pandas as pd
import os
from datetime import date
today = date.today()
print(f'💚 Today is {today}')

💚 Today is 2025-03-18


In [ ]:
path = os.path.dirname(os.getcwd())
print(f'📂 Current working directory: {path}')

/Users/serenekim/Desktop/PhD/meta-wealth_mobility


### OpenAlex API settings:
* First search with the keyword "wealth mobility" [**2673** retrieved articles] (*searched on: 2025-02-20), </br>
* Second search with "intergenerational income mobility" [**1002** retrieved articles] (*searched on: 2025-02-20), </br>
* Third search with "intergenerational wealth mobility" [**175** retrieved articles] (*searched on: 2025-02-20), </br>
* Fourth search with "intergenerational wealth transition" OR "intergenerational income transition" [**277** retrieved articles] (*searched on: 2025-02-20), </br>
* Fifth search with "income+mobility" OR "earning+mobility" [**12692** retrieved articles] (*searched on: 2025-02-26), <br>
and save the results in two separate csv files.


In [ ]:
# Used multiple filter queries to extract articles related to wealth mobility.
# Later combined them into one dataframe and disregarded all duplicates.

mailto = "seorin.kim@vub.be"  
url = "https://api.openalex.org/works"
params = {
    "mailto": mailto,
    # "filter": "title_and_abstract.search:wealth+mobility,type:types/article,type_crossref:journal-article" , # First search query
    # "filter": "title_and_abstract.search:intergenerational+income+mobility,type:types/article,type_crossref:journal-article", # Second search query
    # "filter": "title_and_abstract.search:intergenerational+wealth+mobility,type:types/article,type_crossref:journal-article", # Third search query
    # "filter": "title_and_abstract.search:(intergenerational+wealth+transition) OR (intergenerational+income+transition),type:types/article,type_crossref:journal-article,language:languages/en", # Fourth search query
    "filter": "title_and_abstract.search:(income+mobility) OR (earning+mobility),type:types/article,type_crossref:journal-article,language:languages/en", # Fifth search query
    "per-page": 100,
    "select": "id,doi,publication_year,title,primary_location,abstract_inverted_index,language,open_access",
}

In [4]:
filter_nr = 5 # Update this number to match the filter number in the filter above
cursor = "*"

all_results = [] # The short version of the results
total_papers = 0

# OpenAlex API Pagination
while cursor:
    params["cursor"] = cursor
    response = requests.get(url, params=params)
    if response.status_code != 200:
        print("Error fetching from OpenAlex")
        break
    
    data = response.json()
    total_papers += len(data['results'])  # Count total retrieved papers

    for result in data['results']:
        # Extract DOI and landing page
        id = result.get("id")
        doi_url = result.get("doi")
        landing_page = result.get("primary_location")
        landing_page_url = landing_page.get("landing_page_url") if landing_page else None
        title = result.get("title")
        year = result.get("publication_year")
        abstract_inverted_index = result.get("abstract_inverted_index")
        
        ## Added for the fourth filter
        open_access = result.get("open_access")
        is_oa = open_access.get("is_oa") if open_access else None
        oa_status = open_access.get("oa_status")
        oa_link = open_access.get("oa_url") #I added this for sharing purposes (I forgot this for df4. So I processed it additioanlly in gpt-filter.ipynb)
        language = result.get("language")

        # Store data
        all_results.append({
            "id": id,
            "title": title,
            "year": year,
            "doi": doi_url,
            "landing_page": landing_page_url,
            "abstract_inverted_index": abstract_inverted_index,
            ## Added for the fourth filter
            "language": language,
            "is_oa": is_oa,
            "oa_status": oa_status,
            "oa_link": oa_link
        })

    cursor = data['meta'].get('next_cursor')

print(f"\nTotal papers retrieved from OpenAlex: {total_papers}")

# Save results to a CSV file
with open(f"{path}/data_abstracts/filter{filter_nr}_{today}.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["id", "title", "year", "doi", "landing_page", "abstract_inverted_index", "language", "is_oa", "oa_status", "oa_link"])
    writer.writeheader()
    writer.writerows(all_results)

print(f"\nSearch results saved to filter{filter_nr}_{today}.csv ✅")
print(f"\nToday {today} \n")



Total papers retrieved from OpenAlex: 12692

Search results saved to filter5_2025-02-26.csv ✅

Today 2025-02-26 



# Open Alex Abstracts
**OpenAlex** doesn't provide plaintext abstracts due to legal constraints. Hence, use **PyAlex**. </br>

We'll also use **Semantic Scholar** and **Crossref** since none of these sources are perfect (i.e., Many papers miss their abstracts; For the ones that include abstracts, their quality isn't guaranteed.) </br>

I'll try using [PyAlex](https://github.com/J535D165/pyalex?tab=readme-ov-file) to extract abstracts for all the `id's` from **OpenAlex** based on their `abstract_inverted_index`, and save them in a separate column. Then, I'll also use **Semantic Scholar** and **Crossref** 

In [3]:
from pyalex import Works, Authors, Sources, Institutions, Topics, Publishers

In [6]:
A = pd.read_csv(f"{path}/data_abstracts/filter5_{today}.csv")
B = pd.read_csv(f"{path}/data_abstracts/final_2025-02-24.csv")

In [ ]:
# By PyAlex (Too slow)
# def get_abstracts(data, data_nr):
#     print(f'Starting to get abstract for filter{data_nr}')
#     start = time.time()
#     for id in data['id']:
#         if 'abstract' not in data.columns or data['abstract'].isna().any():
#             w = Works()[id]
#             abstract = w['abstract']
#             data.loc[data['id'] == id, 'abstract'] = abstract
#     print(f'Finished getting abstract for filter{data_nr} in {time.time() - start} seconds') 

In [25]:
# Faster than calling PyAlex. 
# Ref: https://medium.com/towards-data-science/topic-modeling-open-source-research-with-the-openalex-api-5191c7db9156

def undo_inverted_index(inverted_index):
    
    """
    The purpose of the function is to 'undo' and inverted index. It inputs an inverted index and
    returns the original string.
    """

    #create empty lists to store uninverted index
    word_index = []
    words_unindexed = []
    
    #loop through index and return key-value pairs
    for k,v in inverted_index.items(): 
        for index in v: word_index.append([k,index])

    #sort by the index
    word_index = sorted(word_index, key = lambda x : x[1])
    
    #join only the values and flatten
    for pair in word_index:
        words_unindexed.append(pair[0])
    words_unindexed = ' '.join(words_unindexed)
    
    return(words_unindexed)

### Merge the df's

In [8]:
A_in_B = A['id'].isin(B['id']).sum()
print(f'Number of ids in A that exist in B: {A_in_B}')
B_in_A = B['id'].isin(A['id']).sum()
print(f'Number of ids in B that exist in A: {B_in_A}')
print(f'Length of A: {len(A)}')
print(f'Length of B: {len(B)}')

Number of ids in A that exist in B: 1589
Number of ids in B that exist in A: 1584
Length of A: 12692
Length of B: 3713


In [9]:
df5 = A[~A['id'].isin(B['id'])]
print(f'Length of df5: {len(df5)}')

Length of df5: 11103


### Get the abstracts by the function

In [ ]:
# Undo the abstract inverted index from OpenAlex
for i, row in df5.iterrows():
    print(f'Processing row {i}')
    if pd.isna(row['abstract_inverted_index']) == False:
        df5.loc[i, 'abstract'] = undo_inverted_index(eval(df5.loc[i, 'abstract_inverted_index']))
print('✅ Done!')

Processing row 2
Processing row 3
Processing row 4
Processing row 7
Processing row 8
Processing row 10
Processing row 11
Processing row 12
Processing row 13
Processing row 14
Processing row 16
Processing row 17
Processing row 19
Processing row 21
Processing row 22
Processing row 23
Processing row 26
Processing row 27
Processing row 28
Processing row 29
Processing row 30
Processing row 31
Processing row 33
Processing row 37
Processing row 38
Processing row 39
Processing row 40
Processing row 41
Processing row 42
Processing row 44
Processing row 45
Processing row 48
Processing row 49
Processing row 51
Processing row 52
Processing row 53
Processing row 54
Processing row 55
Processing row 56
Processing row 57
Processing row 58
Processing row 61
Processing row 62
Processing row 65
Processing row 69
Processing row 70
Processing row 72
Processing row 75
Processing row 76
Processing row 77
Processing row 78
Processing row 79
Processing row 80
Processing row 81
Processing row 82
Processing row 

In [31]:
# Take abstracts from Semantic Scholar and Crossref
main_semantic_scholar(df5)

Extracting abstracts for 11103 papers
Extracting abstract for https://doi.org/10.2307/1911432


/var/folders/xy/r3gq5vtd7bx6qb966l0fhh9h0000gn/T/ipykernel_16316/1222230421.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['abstract_sm'] = None
/var/folders/xy/r3gq5vtd7bx6qb966l0fhh9h0000gn/T/ipykernel_16316/1222230421.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['authors_sm'] = None


Extracting abstract for https://doi.org/10.1016/0022-0531(78)90101-1
Extracting abstract for https://doi.org/10.1257/jep.11.2.21
Extracting abstract for https://doi.org/10.1126/science.aal4617
Extracting abstract for https://doi.org/10.1006/jeth.1996.0125
Extracting abstract for https://doi.org/10.2307/2547239
Extracting abstract for https://doi.org/10.1111/1468-0297.00296
Extracting abstract for https://doi.org/10.1525/sp.2003.50.1.111
Extracting abstract for https://doi.org/10.1111/1475-4991.00063
Extracting abstract for https://doi.org/10.2307/352279
Extracting abstract for https://doi.org/10.1177/0739456x04264919
Extracting abstract for https://doi.org/10.1080/01603477.1980.11489214
Extracting abstract for https://doi.org/10.1093/oep/gpl014
Extracting abstract for https://doi.org/10.1016/j.jdeveco.2010.01.002
Extracting abstract for https://doi.org/10.2307/2230800
Extracting abstract for https://doi.org/10.1177/0361198105192700108
Extracting abstract for https://doi.org/10.1016/j.j

In [32]:
merged_df = pd.concat([B, df5], ignore_index=True).drop_duplicates(subset=['id']) # Do this after taking all the information from the new df.
print(f'Length of merged_df: {len(merged_df)}')

Length of merged_df: 14760


In [43]:
merged_df.to_csv(f"{path}/data_abstracts/final_{today}_5filters.csv")

# Abstracts from Semantic Scholar and Crossref
🛜 Now we'll extract the abstracts with the doi url. </br>
I found that not all the abstracts are correctly entered in `Open Alex`. So we'll combine other methods.

In [29]:
# Extract DOI from DOI URL
def extract_doi(doi_url):
    return doi_url.split("doi.org/")[-1]

# Semantic Scholar API offers more abstracts. So do sementic scholar first and then crossref.
def get_semantic_scholar_data(doi):
    doi_short = extract_doi(doi)
    url = f"https://api.semanticscholar.org/v1/paper/{doi_short}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        abstract = data.get("abstract", "No abstract available")
        authors = [author["name"] for author in data.get("authors", [])] # It's easier to extract authors from Semantic Scholar.
        return abstract, authors
    else:
        return "Error fetching abstract", []  # Ensure it always returns a tuple


## Semantic Scholar API

In [30]:
def main_semantic_scholar(df):
    print(f"Extracting abstracts for {len(df)} papers")
    count_fail = 0
    start = time.time()
    df['abstract_sm'] = None
    df['authors_sm'] = None

    for doi in df['doi']: #Already exisitng doi's
        print(f"Extracting abstract for {doi}")
        if pd.isna(doi) == False:
            abstract, authors = get_semantic_scholar_data(doi)
            df.loc[df['doi'] == doi, 'abstract_sm'] = abstract
            df.loc[df['doi'] == doi, 'authors_sm'] = "; ".join(authors)
        else:
            count_fail += 1

    print(f"\nAbstracts extracted for {len(df) - count_fail} papers. {count_fail} papers failed to extract abstracts. \nTime taken: {time.time() - start} seconds")


In [ ]:
print(len(merged_df[merged_df['doi'].notna()]) - len(merged_df[(merged_df['abstract_sm'] == 'No abstract available') | (merged_df['abstract_sm'] == 'Error fetching abstract')]))
print(len(merged_df[merged_df['abstract_sm'] == 'No abstract available']))
print(len(merged_df[merged_df['abstract_sm'].isna()]))
# With Semantic Scholars, we have much less abstracts extracted than with OpenAlex (Only 299 papers extracted).

299
0
672


Redoing it doesn't seem to improve the number of non-errors in `abstract`. </br>
</n>Not everything is about "wealth mobitliy" but just the abstract contains "wealth" or "mobility" (not in the sense of social studies but in mobility/transporation studies.)

## Crossref

In [34]:
from crossref.restful import Works

In [35]:
works = Works()

In [36]:
def main_crossref(df):
    count_fail = 0
    start = time.time()
    for doi in df[df['abstract_sm'] == 'Error fetching abstract']['doi'].unique():
        print(f"Retry: {doi} with crossref")
        if pd.isna(doi) == False:
            result = works.doi(doi)
            if result is not None and "abstract" in result.keys():
                df.loc[df['doi'] == doi, 'abstract_sm'] = result['abstract']
                print(result['abstract'])
            else:
                count_fail += 1
                df.loc[df['doi'] == doi, 'abstract_sm'] = None

    print(f"\n{count_fail} doi's with no abstract available. Time taken: {time.time() - start} seconds")            


In [37]:
main_crossref(df5)

Retry: https://doi.org/10.1093/ei/40.3.334 with crossref
<jats:p>Households who work full‐time at minimum wage jobs earn more than the phasein range of the Earned Income Tax Credit (EITC). This means that the EITC mainly acts as a negative income tax. However, most families are eligible to receive the EITC for a relatively short time. Seventy‐four percent of new EITC families will lose their eligibility in two years or less. Sixty‐one percent of families already on the EITC will lose their eligibility in three years or less. EITC families are much more mobile than AFDC families. The main reason that families gain or lose EITC eligibility is changes in earnings.</jats:p>
Retry: https://doi.org/10.1353/eca.2016.0017 with crossref
Retry: https://doi.org/10.1111/j.0013-0427.2004.00367.x with crossref
<jats:p>The paper presents a decomposition of income mobility indices into two basic sources: mobility induced by a change of the income distribution shape, and mobility induced by a reorderin

✅ The main takeaway: </br>
* None of these sources provide reliable abstracts...

🗂️ To-do: </br>
* Filter the papers that truly deal with "intergenerational wealth mobility"

In [44]:
df5 = df5[df5['language'] == 'en']
print(f'Length of merged_df2: {len(df5)}')

Length of merged_df2: 11103


In [60]:
df5_2 = df5[df5['abstract_sm'].notna() | df5['abstract'].notna()].reset_index(drop=True)
print(f'Length of merged_df2: {len(df5_2)}')

Length of merged_df2: 10762


## Domain Filteration (Domain == Social Sciences)
* I'll just use PyAlex since it's simple

In [16]:
data = pd.read_csv(f"{path}/data_abstracts/final_2025-02-24_df1-4.csv")
len(data)

3678

In [21]:
non_list = []
for id in data['id']:
    # if data.loc[data['id'] == id, 'domain'].isna().any():
    w = Works()[id]
    primary_topic = w["primary_topic"]
    if primary_topic:
        domain = primary_topic.get("domain")
        data.loc[data['id'] == id,'domain'] = domain["display_name"]
        if domain['display_name'] != 'Social Sciences':
            non_list.append({'id': id, 'domain': domain})
            print(f"Domain of {id} is not Social Sciences but {domain['display_name']}")

Domain of https://openalex.org/W100816370 is not Social Sciences but Physical Sciences
Domain of https://openalex.org/W114020320 is not Social Sciences but Physical Sciences
Domain of https://openalex.org/W11888872 is not Social Sciences but Physical Sciences
Domain of https://openalex.org/W129213245 is not Social Sciences but Physical Sciences
Domain of https://openalex.org/W1443338233 is not Social Sciences but Health Sciences
Domain of https://openalex.org/W1479912268 is not Social Sciences but Physical Sciences
Domain of https://openalex.org/W1486780719 is not Social Sciences but Physical Sciences
Domain of https://openalex.org/W1504316199 is not Social Sciences but Health Sciences
Domain of https://openalex.org/W1505656569 is not Social Sciences but Physical Sciences
Domain of https://openalex.org/W1512757349 is not Social Sciences but Health Sciences
Domain of https://openalex.org/W1526021467 is not Social Sciences but Physical Sciences
Domain of https://openalex.org/W153460651 i

In [22]:
len(non_list)

649

In [23]:
data.duplicated(subset=['id']).sum()

np.int64(0)

In [24]:
non_list

[{'id': 'https://openalex.org/W100816370',
  'domain': {'id': 'https://openalex.org/domains/3',
   'display_name': 'Physical Sciences'}},
 {'id': 'https://openalex.org/W114020320',
  'domain': {'id': 'https://openalex.org/domains/3',
   'display_name': 'Physical Sciences'}},
 {'id': 'https://openalex.org/W11888872',
  'domain': {'id': 'https://openalex.org/domains/3',
   'display_name': 'Physical Sciences'}},
 {'id': 'https://openalex.org/W129213245',
  'domain': {'id': 'https://openalex.org/domains/3',
   'display_name': 'Physical Sciences'}},
 {'id': 'https://openalex.org/W1443338233',
  'domain': {'id': 'https://openalex.org/domains/4',
   'display_name': 'Health Sciences'}},
 {'id': 'https://openalex.org/W1479912268',
  'domain': {'id': 'https://openalex.org/domains/3',
   'display_name': 'Physical Sciences'}},
 {'id': 'https://openalex.org/W1486780719',
  'domain': {'id': 'https://openalex.org/domains/3',
   'display_name': 'Physical Sciences'}},
 {'id': 'https://openalex.org/W150

In [25]:
# Flatten the data
non_list_df = pd.DataFrame([
    {'id': d['id'], 'domain_id': d['domain']['id'], 'domain_name': d['domain']['display_name']}
    for d in non_list
])

# Remove duplicates if needed
non_list_df = non_list_df.drop_duplicates()

# Display the DataFrame
print(len(non_list_df))

649


In [26]:
non_list_df.to_csv(f"{path}/data_abstracts/non_social_sciences_{today}_df1-4.csv")

In [27]:
data2 = data[~data['id'].isin(non_list_df['id'])].reset_index(drop=True)
data2 = data2.drop_duplicates(subset=['id'])
len(data2)

3029

In [28]:
data2.to_csv(f"{path}/data_abstracts/final1-4_{today}_domain_filtered.csv")

In [32]:
A = pd.read_csv(f"{path}/data_abstracts/filter5_2025-02-26_domain_gpt.csv")
len(A)

7994

In [33]:
A.columns

Index(['Unnamed: 0', 'id', 'title', 'year', 'doi', 'landing_page',
       'abstract_inverted_index', 'language', 'is_oa', 'oa_status', 'oa_link',
       'abstract', 'abstract_sm', 'authors_sm', 'domain', 'sort_gpt_1',
       'sort_gpt_2', 'sort_gpt4o_1', 'sort_gpt4o_2'],
      dtype='object')

In [36]:
print(data2.columns)
print(len(data2))

Index(['Unnamed: 0', 'id', 'title', 'year', 'doi', 'landing_page',
       'abstract_inverted_index', 'abstract', 'abstract_sm', 'authors_sm',
       'sort_gpt_1', 'sort_gpt_2', 'language', 'is_oa', 'oa_link', 'oa_status',
       'sort_gpt4o_1', 'sort_gpt4o_2', 'domain'],
      dtype='object')
3029


In [37]:
# Do this after filling filter5. 
merged_df = pd.concat([A, data2], ignore_index=True).drop_duplicates(subset=['id']) # Do this after taking all the information from the new df.

In [38]:
len(merged_df)

11023